# TextAttack & AllenNLP 

This is an example of testing adversarial attacks from TextAttack on pretrained models provided by AllenNLP. 

In a few lines of code, we load a sentiment analysis model trained on the Stanford Sentiment Treebank and configure it with a TextAttack model wrapper. Then, we initialize the TextBugger attack and run the attack on a few samples from the SST-2 train set.

For more information on AllenNLP pre-trained models: https://docs.allennlp.org/v1.0.0rc3/tutorials/getting_started/using_pretrained_models/

For more information about the TextBugger attack: https://arxiv.org/abs/1812.05271

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QData/TextAttack/blob/master/docs/2notebook/Example_2_allennlp.ipynb)

[![View Source on GitHub](https://img.shields.io/badge/github-view%20source-black.svg)](https://github.com/QData/TextAttack/blob/master/docs/2notebook/Example_2_allennlp.ipynb)

In [ ]:
!pip install allennlp allennlp_models textattack

In [ ]:
!pip install datasets pyarrow transformers --upgrade

In [ ]:
from allennlp.predictors import Predictor
import allennlp_models.classification

import textattack

class AllenNLPModel(textattack.models.wrappers.ModelWrapper):
    def __init__(self):
        self.predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/basic_stanford_sentiment_treebank-2020.06.09.tar.gz")

    def __call__(self, text_input_list):
        outputs = []
        for text_input in text_input_list:
            outputs.append(self.predictor.predict(sentence=text_input))
        # For each output, outputs['logits'] contains the logits where
        # index 0 corresponds to the positive and index 1 corresponds 
        # to the negative score. We reverse the outputs (by reverse slicing,
        # [::-1]) so that negative comes first and positive comes second.
        return [output['logits'][::-1] for output in outputs]

model_wrapper = AllenNLPModel()

In [ ]:
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import TextBuggerLi2018

dataset = HuggingFaceDataset("glue", "sst2", "train")
attack = TextBuggerLi2018(model_wrapper)

results = list(attack.attack_dataset(dataset, indices=range(20)))
for idx, result in enumerate(results):
    print(f'Result {idx}:')
    print(result.__str__(color_method='ansi'))
    print('\n')
print()

textattack: Loading nlp dataset glue, subset sst2, split train.
textattack: Unknown if model of class <class '__main__.AllenNLPModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
/usr/local/lib/python3.6/dist-packages/textattack/constraints/semantics/sentence_encoders/sentence_encoder.py:149: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  embeddings[len(transformed_texts) :]


Result 0:
Negative (95%) --> Positive (93%)

hide new secretions from the parental units 

concealing new secretions from the parental units 



Result 1:
Negative (96%) --> [FAILED]

contains no wit , only labored gags 



Result 2:
Positive (100%) --> [FAILED]

that loves its characters and communicates something rather beautiful about human nature 



Result 3:
Positive (82%) --> [SKIPPED]

remains utterly satisfied to remain the same throughout 



Result 4:
Negative (98%) --> Positive (52%)

on the worst revenge-of-the-nerds clichés the filmmakers could dredge up 

on the pire rеvenge-of-the-nerds clichés the filmmakers could dragging up 



Result 5:
Negative (99%) --> [FAILED]

that 's far too tragic to merit such superficial treatment 



Result 6:
Positive (98%) --> Negative (50%)

demonstrates that the director of such hollywood blockbusters as patriot games can still turn out a small , personal film with an emotional wallop . 

demonstrates that the directors of such tinselt